## **1. Import the necessary libraries** ---

In [1]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers



print("Versions of key libraries")
print("---")
print("tensorflow: ", tf.__version__)
print("numpy:      ", np.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)

Versions of key libraries
---
tensorflow:  2.3.0
numpy:       1.20.3
matplotlib:  3.4.2
sklearn:     0.24.2


## **2.Create a function to plot image without axis** ---

In [2]:
def implt(img):
    plt.figure()
    plt.imshow(img)
    plt.axis('off')

print(implt)

<function implt at 0x000001FF23DE7DC8>


## **3. Set matplotlib to have seaborn plot style**

In [3]:
plt.style.use('seaborn')                   # if want to use the default style, set 'classic'
plt.rcParams['ytick.right']     = True
plt.rcParams['ytick.labelright']= True
plt.rcParams['ytick.left']      = False
plt.rcParams['ytick.labelleft'] = False
plt.rcParams['figure.figsize']  = [7,7]   # Set the figure size to be 7 inch for (width,height)

print("Matplotlib setup completes.")

Matplotlib setup completes.


## **4. Prepare data for training and testing**
---
* Step 1: Load the images
* Step 2: Check the shape and type of the data
* Step 3: Convert the data into float32 and rescale the values from the range of 0\~255 into 0\~1
* Step 4: Retrieve the row size and the column size of each image
* Step 5: Perform one-hot enconding on the labels
* Step 6: Retrieve the number of classes in this problem

In [4]:
from imutils import paths
import os, sys
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def collect_images_and_labels(path_to_images):
    # in seguito, usare https://keras.io/api/preprocessing/image/
    """
        :param path_to_images should be the root folder, in which there is a folder for each label, and the folder's name is
        the label itself
        :return: a list with images and a list with labels
        """
    data = []
    labels = []

    for img_path in list(paths.list_images(path_to_images)):
        # extract the class label from the filename
        label = img_path.split(os.path.sep)[-2]

        # load the input image as (128x128) and preprocess it
        image = load_img(img_path, target_size=(128, 128))
        image = img_to_array(image)
        #image = preprocess_input(image)

        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

    return data, labels

def preprocess_labels(labels):
    """
    :param labels: list of string
    :return: np array of 0/1
    """
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = to_categorical(labels)
    return labels

def tts_split(data, labels):
    (x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)
    (x_train, x_test, y_train, y_test) = (np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test))
    return x_train, x_test, y_train, y_test


PATH_TO_IMAGE = "../data"
MODELS_PATH = "../models"

# Step 1

data, labels = collect_images_and_labels(PATH_TO_IMAGE)
#labels = preprocess_labels(labels)
trDat, tsDat, trLbl, tsLbl = tts_split(data, labels)
# x_train, x_test, y_train, y_test




# Step 2
print("The shape of trDat is", trDat.shape, "and the type of trDat is", trDat.dtype)
print("The shape of tsDat is", tsDat.shape, "and the type of tsDat is", tsDat.dtype)
print("")
print("The shape of trLbl is", trLbl.shape, "and the type of trLbl is", trLbl.dtype)
print("The shape of tsLbl is", tsLbl.shape, "and the type of tsLbl is", tsLbl.dtype)


# Step 3
trDat           = trDat.astype('float32')/255
tsDat           = tsDat.astype('float32')/255

# Step 4
imgrows         = trDat.shape[1]
imgclms         = trDat.shape[2]
channel         = trDat.shape[3]

# Step 5
trLbl           = preprocess_labels(trLbl)
tsLbl           = preprocess_labels(tsLbl)

# Step 6
num_classes     = tsLbl.shape[1]           

E:\anaconda3\envs\detective_mask\lib\site-packages\PIL\Image.py:974: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


The shape of trDat is (6042, 128, 128, 3) and the type of trDat is float32
The shape of tsDat is (1511, 128, 128, 3) and the type of tsDat is float32

The shape of trLbl is (6042,) and the type of trLbl is <U12
The shape of tsLbl is (1511,) and the type of tsLbl is <U12


## *KNN No Hyperparameter Tuning*
___


Build and train KNN model with default parameters

In [5]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA # Dimensionality Reduction


trDat = trDat.reshape(6042, 128*128*3)
tsDat = tsDat.reshape(1511, 128*128*3)

# Initialize KNN model
knn = KNeighborsClassifier()

# Use training data to fit KNN model
knn.fit(trDat, trLbl)
                                                          

KNeighborsClassifier()

In [6]:
%%time
# make prediction on entire test data
predictions_set1 = knn.predict(tsDat)


Wall time: 7.35 s


In [7]:
import pickle
# Save Predictions in a pickle
pickle_out = open("predictions_knn.pickle", "wb")
pickle.dump(predictions_set1, pickle_out)
pickle_out.close()

## **Performance Metrics for No Hyperparameter Tuning**

In [8]:
#Precision, Recall, F1 Score & Classification Report, No Hyperparameter Tuning
print('KNN Precision: %.3f' % precision_score(tsLbl, predictions_set1, average='micro'))
print('KNN Recall: %.3f' % recall_score(tsLbl, predictions_set1, average='micro'))
print('KNN F1 Score: %.3f' % f1_score(tsLbl, predictions_set1, average='micro'))
print("\nNo Hyperparameter Tuning Classification Report\n", classification_report(tsLbl, predictions_set1))


KNN Precision: 0.815
KNN Recall: 0.815
KNN F1 Score: 0.815

No Hyperparameter Tuning Classification Report
               precision    recall  f1-score   support

           0       0.90      0.70      0.79       745
           1       0.76      0.92      0.84       766

   micro avg       0.82      0.82      0.82      1511
   macro avg       0.83      0.81      0.81      1511
weighted avg       0.83      0.82      0.81      1511
 samples avg       0.82      0.82      0.82      1511



# Dimension Reduction

In [10]:
# Dimensionality Reduction with Principal Component Analysis (PCA)
pca = PCA(0.90) # Preserve 90% of the variance

X_transformed = pca.fit_transform(trDat) # Fit the pca transform with X_train
X_test_transformed = pca.transform(tsDat) # Apply transform to X_test

# Training set shape after Principal Component Analysis form
print(X_transformed.shape)

# Original Training Set Shape
# Notice we lose 3,943 features using PCA, while preserving 90% variance
print(trDat.shape)

(6042, 189)
(6042, 49152)


In [12]:
# Initialize KNN model for PCA instance
knn_pca = KNeighborsClassifier()

# Use training data to fit KNN model with transformed X_train
knn_pca.fit(X_transformed, trLbl)

KNeighborsClassifier()

In [13]:
%%time
# make prediction on entire test data
predictions_set_pca = knn_pca.predict(X_test_transformed)
print("Accuracy Train PCA:", accuracy_score(tsLbl, predictions_set_pca))
print('KNN Recall: %.3f' % recall_score(tsLbl, predictions_set_pca, average='micro'))
print('KNN F1 Score: %.3f' % f1_score(tsLbl, predictions_set_pca, average='micro'))

Accuracy Train PCA: 0.8391793514228988
KNN Recall: 0.839
KNN F1 Score: 0.839
Wall time: 272 ms


# KNN Hyperparameter Tuning with GridSearchCV

In [15]:
# New KNN instance model
knn2 = KNeighborsClassifier()

# Dictionary of parameter values we are testing performance for
param_grid = {'n_neighbors': [5, 7, 9, 11], 'p': [2, 4, 6, 8, 10, 20, 50], 'metric': ['euclidean', 'manhattan', 'minkowski']}

# Test all parameter combinations in param_grid
knn_gscv = GridSearchCV(knn2, param_grid, scoring='f1_micro', cv=5, verbose=3, n_jobs=-1)

In [ ]:
# Fit model to data
knn_gscv.fit(trDat, trLbl)

Fitting 5 folds for each of 84 candidates, totalling 420 fits


In [ ]:
# Get GridSearchCV's top performing parameters and score

# Best Params: n_neighbors=5, distance=manhattan, p=2
best_params_ = knn_gscv.best_params_

# Best Score 82.7%
knn_gscv.best_score

In [6]:
best_params = {'weights': 'distance', 'n_neighbors': 2, 'metric': 'manhattan'}

In [7]:
# create new a knn model with best params
b_knn = KNeighborsClassifier(**best_params)

#fit model to data
b_knn.fit(trDat, trLbl)

# make prediction on entire train data
train_pred = b_knn.predict(trDat)

# make prediction on entire test data
y_pred = b_knn.predict(tsDat)

print('Accuracy Train: %.3f' % accuracy_score(trLbl, train_pred))
print('Accuracy Test: %.3f' % accuracy_score(tsLbl, y_pred))
print('KNN Recall: %.3f' % recall_score(tsLbl, y_pred, average='micro'))
print('KNN F1 Score: %.3f' % f1_score(tsLbl, y_pred, average='micro'))
print("\nClassification Report\n", classification_report(tsLbl, y_pred))

Accuracy Train: 1.000
Accuracy Test: 0.809
KNN Recall: 0.809
KNN F1 Score: 0.809

Classification Report
               precision    recall  f1-score   support

           0       0.84      0.76      0.80       745
           1       0.78      0.86      0.82       766

   micro avg       0.81      0.81      0.81      1511
   macro avg       0.81      0.81      0.81      1511
weighted avg       0.81      0.81      0.81      1511
 samples avg       0.81      0.81      0.81      1511



In [8]:
# Dimensionality Reduction with Principal Component Analysis (PCA)
pca = PCA(0.90) # Preserve 90% of the variance

X_transformed = pca.fit_transform(trDat) # Fit the pca transform with X_train
X_test_transformed = pca.transform(tsDat) # Apply transform to X_test

# Training set shape after Principal Component Analysis form
print(X_transformed.shape)

# Original Training Set Shape
# Notice we lose 3,943 features using PCA, while preserving 90% variance
print(trDat.shape)

(6042, 189)
(6042, 49152)


In [11]:
# Initialize KNN model for PCA instance
knn_pca2 = KNeighborsClassifier(**best_params)

# Use training data to fit KNN model with transformed X_train
knn_pca2.fit(X_transformed, trLbl)

KNeighborsClassifier(metric='manhattan', n_neighbors=2, weights='distance')

In [12]:
%%time
# make prediction on entire test data
predictions_set_pca = knn_pca2.predict(X_test_transformed)
print("Accuracy Train PCA:", accuracy_score(tsLbl, predictions_set_pca))
print('KNN Recall: %.3f' % recall_score(tsLbl, predictions_set_pca, average='micro'))
print('KNN F1 Score: %.3f' % f1_score(tsLbl, predictions_set_pca, average='micro'))

Accuracy Train PCA: 0.7875579086697552
KNN Recall: 0.788
KNN F1 Score: 0.788
Wall time: 2.71 s
